# Long-term Chat Memory Agent with AutoGen
---

This example demonstrates how to create a long-term chat memory agent using the AutoGen framework. The agent is designed to remember past interactions and provide contextually relevant responses over time.

- Reference: https://microsoft.github.io/autogen/stable//user-guide/agentchat-user-guide/memory.html

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_core.memory import ListMemory, MemoryContent, MemoryMimeType
from autogen_core.memory import Memory, MemoryContent, MemoryMimeType

from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from IPython.display import display, Markdown

<br>

## 🧪 1. Preparation and Define the Agentic Architecture
---
### Initialize the Memory

You can implement your own custom memory store based on the memory protocol. For example, you can implement a custom memory store that uses a vector database to store and retrieve information, or a memory store that uses machine learning models to generate personalized responses based on user preferences, etc. This example implements a memory store based on FAISS, which is not yet supported by AutoGen (as of April 2025). 

> **Note: There may be errors in the implementation, so please use it only for reference.**

In [ ]:
from src.faiss import FAISSVectorMemoryConfig, FAISSVectorMemory

MEMORY_DB = "list"  # or "list"
assert MEMORY_DB in ["faiss", "list"], f"Unsupported memory db: {MEMORY_DB}"

if MEMORY_DB == "faiss":
    print(
        "=== Using Custom FAISS vector memory. You can refer to src/faiss.py for more details."
    )
    user_memory = FAISSVectorMemory(
        config=FAISSVectorMemoryConfig(
            emb_model_name="text-embedding-3-large",
            dimension=3072,
            use_gpu=False,
            top_k=2,
            score_threshold=0.3,
        )
    )
elif MEMORY_DB == "list":
    print("=== Using Built-in List memory")

    user_memory = ListMemory()

await user_memory.clear()

### Add some initial memory content

In [ ]:
async def add_memory(user_memory, content, metadata, mime_type=MemoryMimeType.TEXT):
    await user_memory.add(
        MemoryContent(
            content=content,
            mime_type=mime_type,
            metadata=metadata,
        )
    )

Baseline: You can add memory with the text format.

In [ ]:
await add_memory(
    user_memory,
    "Daekeun provides AIML technology support. Hyo is a big fan of Microsoft.",
    {"user_id": "1"},
    MemoryMimeType.TEXT,
)
await add_memory(
    user_memory,
    "Daekeun is interested in AutoGen and Semantic Kernel",
    {"user_id": "1"},
    MemoryMimeType.TEXT,
)

You can also add memory with JSON format. The memory content can be a list of dictionaries.

In [ ]:
from azure_genai_utils.tools import BingSearch

web_search_tool = BingSearch(
    max_results=2,
    locale="en-US",
    include_news=False,
    include_entity=False,
    format_output=False,
)

# Define the tools to be used in the state graph
query = "What is AutoGen's main feature?"
results = web_search_tool.invoke(query)

for i in range(1, len(results)):
    await add_memory(user_memory, results[i], {"user_id": "1"}, MemoryMimeType.JSON)

### Define the tools

In [ ]:
from autogen_core.tools import FunctionTool


def arxiv_search(query: str, max_results: int = 2) -> list:  # type: ignore[type-arg]
    """
    Search Arxiv for papers and return the results including abstracts.
    """
    import arxiv

    client = arxiv.Client()
    search = arxiv.Search(
        query=query, max_results=max_results, sort_by=arxiv.SortCriterion.Relevance
    )

    results = []
    for paper in client.results(search):
        results.append(
            {
                "title": paper.title,
                "authors": [author.name for author in paper.authors],
                "published": paper.published.strftime("%Y-%m-%d"),
                "abstract": paper.summary,
                "pdf_url": paper.pdf_url,
            }
        )

    # # Write results to a file
    # with open('arxiv_search_results.json', 'w') as f:
    #     json.dump(results, f, indent=2)

    return results


async def get_weather(city: str, units: str = "imperial") -> str:
    if units == "imperial":
        return f"The weather in {city} is 73 °F and Sunny."
    elif units == "metric":
        return f"The weather in {city} is 23 °C and Sunny."
    else:
        return f"Sorry, I don't know the weather in {city}."


arxiv_search_tool = FunctionTool(
    arxiv_search,
    description="Search Arxiv for papers related to a given topic, including abstracts",
)

In [ ]:
model_client = AzureOpenAIChatCompletionClient(
    model="gpt-4o-mini",
    temperature=0.1,
    max_tokens=500,
    top_p=1,
)

assistant_agent = AssistantAgent(
    name="assistant_agent",
    description="An assistant agent that can answer questions and provide information.",
    model_client=model_client,
    tools=[get_weather, arxiv_search_tool],
    memory=[user_memory],
)

<br>

## 🧪 2. Run the agent with a task.
---

In [ ]:
stream = assistant_agent.run_stream(task="What is AutoGen?")
await Console(stream)

In [ ]:
message = await assistant_agent._model_context.get_messages()
display(Markdown(message[-1].content))

### This below code cell checks if the memory is working.

In [ ]:
stream = assistant_agent.run_stream(task="Who is Daekeun?")
await Console(stream)

In [ ]:
message = await assistant_agent._model_context.get_messages()
display(Markdown(message[-1].content))